<a href="https://colab.research.google.com/github/dangdang2222/LG_Aimers/blob/main/finalmodel_featureselectiony10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2개로 가기(xgboost, randomforest)
1.이상치 제거
-->1% 클리핑(원래는 각 X 피쳐 속성 다 고려해야하지만 시간이 없으니깐...)
2. 인희님이 feature 만드신거 의미있어보이므로 일단은 feature 만들어놓고 생각
--> feature 만드는건 바로 아래 글 참조
feature 추가해서 만들고, 해당 feature 만드는데 사용되었던 feature은 일단 drop x,
+반드시 통과여부 drop하기******
3. rough하게 준 모델 parameter tuning for each y feature
xgboost and randomforest
xgboost initial 파라미터
xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,colsample_bytree=1, max_depth=7)
randomforest initial 파라미터
RandomForestRegressor(criterion="squared_error", random_state=2022, n_jobs=-1, n_estimators=300, max_depth=150, min_samples_leaf=2, min_samples_split=2)
)

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import random
import os
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,13): # ignore 'ID'
        rmse = mean_squared_error(np.array(gt)[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(np.array(gt)[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [4]:
def lg_nrmse_one(gt, preds):
    # Y_08의 경우는 가중치 1.2 부여. 8 이후부터는 모두 가중치 1.0
    all_nrmse = []
    rmse = mean_squared_error(np.array(gt), preds, squared=False)
    nrmse = rmse/np.mean(np.abs(np.array(gt)))
    all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse)
    return score

#얘를 쓸것!!!

In [5]:
train_df = pd.read_csv('/content/drive/MyDrive/LG Aimer/train.csv').drop(columns="ID")
train_x = train_df.filter(regex='X') # Input : X Featrue
train_x=train_x.drop(columns=['X_04','X_23','X_47','X_48'])
train_y = train_df.filter(regex='Y') # Output : Y Feature



#Y_10

In [6]:
train_y10 = train_y['Y_10']
train_y10

0       -22.769
1       -22.574
2       -22.169
3       -21.765
4       -21.072
          ...  
39602   -22.772
39603   -24.674
39604   -22.577
39605   -24.066
39606   -23.257
Name: Y_10, Length: 39607, dtype: float64

In [7]:
train_y10=pd.DataFrame(train_y10, index= train_y.index)

#클리핑

In [8]:
from sklearn.model_selection import train_test_split

# -----------------------------------
# 열마다 학습 데이터의 1%, 99% 지점을 확인
p01 = train_x.quantile(0.01)
p99 = train_x.quantile(0.99)

# 1％점 이하의 값은 1%점으로, 99%점 이상의 값은 99%점으로 클리핑
train_x = train_x.clip(p01, p99, axis=1)


#핀 휜 정도 피쳐 만들기

In [9]:
# 커넥터 핀 휨 
train_x['X__21'] = (train_x['X_24'] - train_x['X_12'])
train_x['X__22'] = (train_x['X_25'] - train_x['X_12'])
train_x['X__23'] = (train_x['X_26'] - train_x['X_12'])
train_x['X__24'] = (train_x['X_27'] - train_x['X_12'])
train_x['X__25'] = (train_x['X_28'] - train_x['X_12'])
train_x['X__26'] = (train_x['X_29'] - train_x['X_12'])
#이후, RFE 상에서 혹여나 새로운 변수를 만드는데 기여한 기존 변수들(12, 24,25,26,27,28,29)가 important하게 잡힐 수 있으므로 일단은 놔둡니다.
#또한, 현재 train_x 상에서는 통과여부 변수들은 모두 drop된 상태입니다.


In [10]:
train_x.columns

Index(['X_01', 'X_02', 'X_03', 'X_05', 'X_06', 'X_07', 'X_08', 'X_09', 'X_10',
       'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17', 'X_18', 'X_19',
       'X_20', 'X_21', 'X_22', 'X_24', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29',
       'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35', 'X_36', 'X_37', 'X_38',
       'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44', 'X_45', 'X_46', 'X_49',
       'X_50', 'X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56', 'X__21',
       'X__22', 'X__23', 'X__24', 'X__25', 'X__26'],
      dtype='object')

RFE 돌리기

#XGBoost

In [11]:
import xgboost as xgb
regressor1= xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,
                           colsample_bytree=1, max_depth=7)


In [12]:
from sklearn.feature_selection import RFE
n_features_to_select=1
rfe1=RFE(regressor1, n_features_to_select=n_features_to_select)

rfe1.fit(train_x, train_y10) 
#다른 데에서 RFE fit할때는 전체 데이터로 하던거 같더라구요
from operator import itemgetter
features = train_x.columns.to_list()
rank_xgb = []
for x,y in (sorted(zip(rfe1.ranking_,features), key=itemgetter(0))):
  print(x,y)
  rank_xgb.append(y)



[01:09:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:10:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:11:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:11:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:12:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:12:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:13:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:14:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

#이후 최적의 경우가 어떨때인지를 본다

In [13]:
score_list=[]
for i in range(len(rank_xgb)):
    cols_forxgb = rank_xgb[:(i+1)]
    train_x_xgb = train_x[cols_forxgb]
    x_train,x_valid, y_train,y_valid = train_test_split(train_x_xgb,train_y10,test_size=0.3) 

    model_xgb = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,colsample_bytree=1, max_depth=7)
    model_xgb.fit(x_train, y_train) ##xgboost error....?
    y_predict = model_xgb.predict(x_valid)
    score = lg_nrmse_one(y_valid, y_predict)
    score_list.append(score)

#여기서 최대인 애 인덱스 찾으면 해당 경우가 최적


[01:31:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:31:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:31:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:31:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:31:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:31:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:31:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:31:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [14]:
score_list

[0.04872706567341282,
 0.04822684514424302,
 0.04695671566651021,
 0.047417408609239785,
 0.04562948279372415,
 0.04611432898774085,
 0.04531087380185091,
 0.04591128187921033,
 0.04537809953174844,
 0.04538388003589337,
 0.04649316238242201,
 0.04579281572047778,
 0.045830128518713945,
 0.04516097951288025,
 0.045778787362362024,
 0.04597259619721921,
 0.04534656019293334,
 0.04459492355698042,
 0.04543009694689347,
 0.04525672089078208,
 0.04524181568604083,
 0.0446586343831496,
 0.046685245227728184,
 0.046635411324946356,
 0.04623262497171936,
 0.04740953477548054,
 0.04624449021576787,
 0.0463822369406331,
 0.046243702210013185,
 0.045429651899735475,
 0.04620228907432324,
 0.04489244779164929,
 0.045522708742988575,
 0.046348411364228584,
 0.045906161104053524,
 0.045994133663844174,
 0.046085142773598424,
 0.04652783015994039,
 0.04568049839036085,
 0.04536729170000848,
 0.046930001092412285,
 0.045822363856334376,
 0.04535873352602035,
 0.046699133014341664,
 0.0464587356372503

In [15]:
tmp = min(score_list)
index = score_list.index(tmp)

print(index)

17


#Randomforest

In [ ]:
regressor2= RandomForestRegressor(criterion="squared_error", random_state=2022, n_jobs=-1, n_estimators=300, max_depth=150, min_samples_leaf=2, min_samples_split=2)
from sklearn.feature_selection import RFE
n_features_to_select=1
rfe2=RFE(regressor2, n_features_to_select=n_features_to_select)

rfe2.fit(train_x, train_y08)
from operator import itemgetter
features = train_x.columns.to_list()
rank_rf = []
for x,y in (sorted(zip(rfe2.ranking_,features), key=itemgetter(0))):
  print(x,y)
  rank_rf.append(y)



/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_rfe.py:283: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_rfe.py:283: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_rfe.py:283: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_rfe.py:283: DataConversionWarning: A column-vector y was passed when a 1d array was expe

1 X_49
2 X_08
3 X_09
4 X_07
5 X_56
6 X_03
7 X_51
8 X_54
9 X_55
10 X_53
11 X_05
12 X_52
13 X_20
14 X_50
15 X_39
16 X_21
17 X_22
18 X_19
19 X_32
20 X_45
21 X_38
22 X_36
23 X__26
24 X_35
25 X_37
26 X_34
27 X_16
28 X_33
29 X_43
30 X__25
31 X_42
32 X_31
33 X_40
34 X_25
35 X_15
36 X_44
37 X_13
38 X_30
39 X_41
40 X__24
41 X_01
42 X__23
43 X_28
44 X_12
45 X_14
46 X_27
47 X__21
48 X_06
49 X_29
50 X_17
51 X_26
52 X__22
53 X_18
54 X_24
55 X_46
56 X_02
57 X_11
58 X_10


In [ ]:
score_list=[]
for i in range(len(rank_rf)):
  cols_forrf = rank_rf[:(i+1)]
  train_x_rf = train_x[cols_forrf]
  x_train,x_valid, y_train,y_valid = train_test_split(train_x_rf,train_y08,test_size=0.3) 
  model_rf =RandomForestRegressor(criterion="squared_error", random_state=2022, n_jobs=-1, n_estimators=300, max_depth=150, min_samples_leaf=2, min_samples_split=2)
  model_rf.fit(x_train,y_train)
  y_predict = model_rf.predict(x_valid)
  score = lg_nrmse_one(y_valid, y_predict)
  score_list.append(score)

#여기서 최대인 애 인덱스 찾으면 해당 경우가 최적


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DataConv

In [ ]:
tmp = max(score_list)
index = score_list.index(tmp)
print(index)

여기까지가 feature selection part!
추가로 parameter tuning 후 최종적인 모델 build!